In [2]:
import pandas as pd
import re
nums_list = ["1", "5", "10","20", "50"]
tpc_types = ["tpc-h/"]

In [3]:
def average_query_metrics(paths, columns_to_use, output_path):
    # Read, filter, and clean data
    dfs = [
        pd.read_csv(path, comment='/')[columns_to_use]
        .assign(query=lambda df: df['query'].astype(str).str.strip())  # Convert to string and strip
        for path in paths
    ]
    
    # Merge data and calculate average by query
    all_data = pd.concat(dfs)
    average_metrics = all_data.groupby('query', as_index=False).mean(numeric_only=True)
    average_metrics = average_metrics.round(2)
    
    # Add column for sorting by number in query
    average_metrics['query_number'] = average_metrics['query'].apply(lambda x: int(re.search(r'\d+', x).group()))
    
    # Sort by query number
    average_metrics = average_metrics.sort_values(by='query_number').drop(columns=['query_number'])
    
    # Write to file
    average_metrics.to_csv(output_path, index=False)
    
    return average_metrics

In [4]:
for tpc_type in tpc_types:
    for nums in nums_list:
        # average_path_1 = tpc_type+nums+"gb/cleaned/run1_cleaned.csv"
        average_path_2 = tpc_type+nums+"gb/cleaned/run2_cleaned.csv"
        average_path_3 = tpc_type+nums+"gb/cleaned/run3_cleaned.csv"

        average_output_path = "result/"+tpc_type+"average_query_case_"+nums+"gb.csv"
        # Đường dẫn đến 3 file CSV
        average_paths = [
            average_path_2,average_path_3
        ]

        columns_to_use = [
            'query',
            'time (ms)'
        ]
        # Process average metrics
        average_metrics = average_query_metrics(
            average_paths, 
            columns_to_use, 
            average_output_path
        )
